# **Project Sistem Rekomendasi**

## Tujuan dan Manfaat Proyek

**Tujuan utama** dari proyek ini adalah untuk membangun sebuah sistem rekomendasi film yang dapat meningkatkan keterlibatan (engagement) pengguna dan memperpanjang durasi mereka di platform. Kami ingin menyediakan rekomendasi yang sangat personal dan akurat, yang terasa seperti kurasi dari seorang ahli film.

**Manfaat langsung** dari pencapaian tujuan ini adalah:

* Peningkatan Retensi Pengguna: Pengguna yang merasa platform memahami selera mereka cenderung akan kembali lagi dan tidak beralih ke layanan lain.

* Optimalisasi Konten: Wawasan dari model dapat membantu tim konten dalam mengambil keputusan yang didukung data tentang film apa yang harus ditambahkan ke katalog.

* Peluang Monetisasi: Durasi tontonan yang lebih lama dan tingkat kunjungan yang lebih sering membuka peluang untuk pendapatan dari iklan atau peningkatan langganan.



#Import Data

In [ ]:
# Import pustaka yang diperlukan
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re

# Import pustaka yang diperlukan
import pandas as pd
from fastai.collab import *
from fastai.tabular.all import *
from pathlib import Path

In [ ]:
!git clone https://github.com/Andishafira/Dicoding.git

Cloning into 'Dicoding'...
remote: Enumerating objects: 104, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 104 (delta 24), reused 38 (delta 8), pack-reused 31 (from 2)
Receiving objects: 100% (104/104), 158.92 MiB | 27.20 MiB/s, done.
Resolving deltas: 100% (27/27), done.
Updating files: 100% (25/25), done.


In [ ]:
import pandas as pd

df_movie_path = "/content/Dicoding/Belajar Machine Learning Terapan/Tugas 2_Sistem Rekomendasi/dataset/movies.csv"
df_ratings_path = "/content/Dicoding/Belajar Machine Learning Terapan/Tugas 2_Sistem Rekomendasi/dataset/ratings.csv"
df_tags_path = "/content/Dicoding/Belajar Machine Learning Terapan/Tugas 2_Sistem Rekomendasi/dataset/tags.csv"
df_links_path = "/content/Dicoding/Belajar Machine Learning Terapan/Tugas 2_Sistem Rekomendasi/dataset/links.csv"

df_movie = pd.read_csv(df_movie_path)
df_ratings = pd.read_csv(df_ratings_path)
df_tags = pd.read_csv(df_tags_path)
df_links = pd.read_csv(df_links_path)

In [ ]:
# Menampilkan 5 baris pertama dari movies_df
print("--- 5 baris pertama dari movies.csv ---")
print(df_movie.head())

# Menampilkan informasi kolom dan tipe data dari movies_df
print("\n--- Informasi dataframe movies_df ---")
print(df_movie.info())

# Menampilkan 5 baris pertama dari ratings_df
print("\n--- 5 baris pertama dari ratings.csv ---")
print(df_ratings.head())

# Menampilkan informasi kolom dan tipe data dari ratings_df
print("\n--- Informasi dataframe ratings_df ---")
print(df_ratings.info())

# Memeriksa jumlah nilai unik pada kolom 'genres'
print("\n--- Jumlah genre unik ---")
print(len(df_movie['genres'].unique()))

# Mengambil 10 genre teratas
print("\n--- 10 genre teratas ---")
# Kita akan memisahkan genre terlebih dahulu untuk mendapatkan gambaran yang lebih akurat
all_genres = df_movie['genres'].str.split('|', expand=True).stack().reset_index(level=1, drop=True)
print(all_genres.value_counts().head(10))

--- 5 baris pertama dari movies.csv ---
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  

--- Informasi dataframe movies_df ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory 

#Preprocessing Data

In [ ]:
# Gabungkan data ratings dan movies
ratings = df_ratings.merge(df_movie)

In [ ]:
ratings.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [ ]:
#split ratings to train df and test df (80:20)
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(ratings, test_size=0.2, random_state=42)

#Train Model

In [ ]:
# --- Model Collaborative Filtering (fastai) ---
# Persiapan data loaders
dls = CollabDataLoaders.from_df(
    train_df,
    user_name='userId',
    item_name='title',
    rating_name='rating',
    bs=64
)

# Latih model
learn = collab_learner(dls, n_factors=50, y_range=(0.5, 5.5), metrics=rmse)
learn.fit_one_cycle(5, 5e-3)

# --- Model Content-based (berbasis genre) ---
# Menggabungkan semua genre menjadi satu kolom string untuk vektorisasi
# genres_df = df_movie.drop(columns=['movieId','title'])
# genres_df['genres'] = genres_df.iloc[:, 1:].apply(
#     lambda row: '|'.join([col for col, val in row.items() if val == 1]), axis=1
# )

# Use the existing 'genres' column from df_movie directly
genres_data = df_movie['genres']

# Inisialisasi TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tfidf_vectorizer = TfidfVectorizer(stop_words='english')
# Use the corrected genres_data for fitting and transforming
tfidf_matrix = tfidf_vectorizer.fit_transform(genres_data)

# Hitung Cosine Similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

epoch,train_loss,valid_loss,_rmse,time
0,0.722827,0.809334,0.899630,00:17
1,0.401113,0.778353,0.882243,00:14
2,0.246760,0.788836,0.888165,00:21
3,0.171190,0.793321,0.890686,00:14
4,0.153360,0.794356,0.891267,00:16


##Membuat Model Hybrid

In [ ]:
def get_hybrid_recommendations(user_id, num_recommendations=10, collaborative_weight=0.6, content_weight=0.4):
    """
    Memberikan rekomendasi film hibrida untuk pengguna tertentu.

    Fungsi ini menggabungkan:
    1. Prediksi rating dari model Collaborative Filtering.
    2. Kemiripan genre dari model Content-based Filtering.

    Args:
        user_id (int): ID pengguna.
        num_recommendations (int): Jumlah rekomendasi yang diinginkan.
        collaborative_weight (float): Bobot untuk skor collaborative (prediksi rating).
        content_weight (float): Bobot untuk skor content-based (kemiripan genre).

    Returns:
        pd.DataFrame: Daftar film yang direkomendasikan dengan skor hibrida.
    """
    # 1. Mendapatkan daftar film yang belum pernah ditonton pengguna
    user_rated_movies = train_df[train_df['userId'] == user_id]['title'].tolist()
    all_movies_titles = df_movie['title'].tolist()
    unrated_movies = [title for title in all_movies_titles if title not in user_rated_movies]

    # Pilih 100 film acak dari yang belum ditonton untuk diprediksi (untuk efisiensi)
    unrated_movies_sample = pd.Series(unrated_movies).tolist()

    # 2. Menggunakan model Collaborative untuk memprediksi rating
    user_tensor = torch.tensor([dls.classes['userId'].o2i[user_id]] * len(unrated_movies_sample))
    movie_titles_tensor = torch.tensor([dls.classes['title'].o2i[title] for title in unrated_movies_sample])

    # Concatenate user and movie tensors
    collab_input = torch.stack([user_tensor, movie_titles_tensor], dim=1)

    with torch.no_grad():
        collab_preds = learn.model.forward(collab_input)
    collab_preds = collab_preds.numpy()

    collab_df = pd.DataFrame({'title': unrated_movies_sample, 'predicted_rating': collab_preds.flatten()}) # Flatten to match the shape

    # 3. Mendapatkan film favorit pengguna (sebagai "benih" untuk model Content-based)
    # Kita ambil 5 film dengan rating tertinggi dari pengguna
    user_top_movies = train_df[(train_df['userId'] == user_id) & (train_df['rating'] >= 4)].sort_values(by='rating', ascending=False).head(5)

    if user_top_movies.empty:
        print("Pengguna ini belum memiliki rating film tinggi. Menggunakan rekomendasi Collaborative saja.")
        # Add content similarity scores of 0 for all recommendations if no top movies
        collab_df['content_sim_score'] = 0.0
        collab_df['hybrid_score'] = collaborative_weight * collab_df['predicted_rating']
        return collab_df.sort_values(by='hybrid_score', ascending=False).head(num_recommendations)


    seed_movie_title = user_top_movies['title'].iloc[0]

    # 4. Menghitung skor kemiripan konten untuk re-ranking
    # Dapatkan indeks film 'seed'
    seed_movie_idx = df_movie[df_movie['title'] == seed_movie_title].index[0]

    # Dapatkan skor kemiripan dari film seed dengan semua film lain
    content_sim_scores = cosine_sim[seed_movie_idx]

    # Gabungkan dengan dataframe prediksi
    hybrid_recs = collab_df.merge(df_movie[['title']], on='title')
    hybrid_recs['content_sim_score'] = hybrid_recs['title'].apply(
        lambda x: content_sim_scores[df_movie[df_movie['title'] == x].index[0]]
    )

    # 5. Menggabungkan skor untuk mendapatkan skor hibrida
    hybrid_recs['hybrid_score'] = (
        collaborative_weight * hybrid_recs['predicted_rating'] +
        content_weight * (hybrid_recs['content_sim_score'] * 5) # Kalikan dengan 5 agar skalanya mirip
    )

    # Urutkan berdasarkan skor hibrida dan tampilkan hasilnya
    final_recs = hybrid_recs.sort_values(by='hybrid_score', ascending=False).head(num_recommendations)
    return final_recs[['title', 'hybrid_score', 'predicted_rating', 'content_sim_score']]

##Contoh Testing

In [ ]:
# Contoh penggunaan:
user_id_to_recommend = 197
print(f"--- Rekomendasi Hibrida untuk Pengguna {user_id_to_recommend} ---")
recommendations = get_hybrid_recommendations(user_id_to_recommend)
print(recommendations)

--- Rekomendasi Hibrida untuk Pengguna 197 ---
                                    title  hybrid_score  predicted_rating  \
189                         Clerks (1994)      4.858077          4.763462   
7569         Louis C.K.: Hilarious (2010)      4.834403          4.724005   
3298       Dirty Rotten Scoundrels (1988)      4.797059          4.661765   
147             Living in Oblivion (1995)      4.776928          4.628214   
974             This Is Spinal Tap (1984)      4.732128          4.553546   
2962  Planes, Trains & Automobiles (1987)      4.718445          4.530741   
1060               Raising Arizona (1987)      4.680098          4.466830   
8208         Louis C.K.: Oh My God (2013)      4.675273          4.458788   
7570         Louis C.K.: Chewed Up (2008)      4.661955          4.436591   
378             Dazed and Confused (1993)      4.660936          4.434893   

      content_sim_score  
189                 1.0  
7569                1.0  
3298                1.0  
1

#Evaluasi Model

In [ ]:
def evaluate_hybrid_recommender(model, ratings_df, test_df, k=10, min_rating=4.0):
    """
    Mengevaluasi sistem rekomendasi hibrida menggunakan metrik Precision@k dan Recall@k.

    Args:
        model: Model fastai yang sudah dilatih.
        ratings_df (pd.DataFrame): Dataframe rating lengkap.
        k (int): Jumlah item teratas yang akan dipertimbangkan untuk rekomendasi.
        min_rating (float): Rating minimum untuk dianggap "relevan".

    Returns:
        tuple: Precision@k dan Recall@k rata-rata.
    """
    all_users = test_df['userId'].unique()
    all_movies = ratings_df['title'].unique()

    precision_scores = []
    recall_scores = []

    # Ambil sampel beberapa pengguna untuk evaluasi yang efisien
    sample_users = pd.Series(all_users).sample(n=min(50, len(all_users)), random_state=42)

    for user_id in sample_users:
        # Dapatkan film yang sudah dirating pengguna
        user_ratings = test_df[test_df['userId'] == user_id]

        # Pisahkan film relevan (rating tinggi) dari film yang tidak relevan
        relevant_movies = user_ratings[user_ratings['rating'] >= min_rating]['title'].tolist()
        print(f"Pengguna {user_id} memiliki {len(relevant_movies)} film relevan.")
        # Dapatkan rekomendasi hibrida
        # Karena ini simulasi, kita gunakan fungsi rekomendasi sebelumnya
        recommendations = get_hybrid_recommendations(user_id, num_recommendations=k)
        print(f"Rekomendasi untuk pengguna {user_id}: {recommendations['title'].tolist()}")
        if recommendations is None:
            continue

        recommended_movies = recommendations['title'].tolist()

        # Hitung irisan antara film yang direkomendasikan dan film yang relevan
        hits = len(set(recommended_movies) & set(relevant_movies))
        print(f"Pengguna {user_id} memiliki {hits} hits.")
        # Hitung Precision@k
        precision = hits / len(recommended_movies) if len(recommended_movies) > 0 else 0
        precision_scores.append(precision)

        # Hitung Recall@k
        recall = hits / len(relevant_movies) if len(relevant_movies) > 0 else 0
        recall_scores.append(recall)

    # Hitung rata-rata Precision dan Recall
    avg_precision = sum(precision_scores) / len(precision_scores) if precision_scores else 0
    avg_recall = sum(recall_scores) / len(recall_scores) if recall_scores else 0

    return avg_precision, avg_recall



In [ ]:
# Uji coba dengan k=10 dan rating relevan >= 4
avg_prec, avg_rec = evaluate_hybrid_recommender(learn, ratings, test_df, k=10, min_rating=4.0)
print(f"Rata-rata Precision@{10}: {avg_prec:.4f}")
print(f"Rata-rata Recall@{10}: {avg_rec:.4f}")

Pengguna 428 memiliki 4 film relevan.
Rekomendasi untuk pengguna 428: ['Juno (2007)', 'Harold and Maude (1971)', 'The Artist (2011)', 'Postman, The (Postino, Il) (1994)', 'Love Actually (2003)', 'Roman Holiday (1953)', 'Philadelphia Story, The (1940)', 'Moonrise Kingdom (2012)', 'Adaptation (2002)', 'Hannah and Her Sisters (1986)']
Pengguna 428 memiliki 0 hits.
Pengguna 582 memiliki 9 film relevan.
Rekomendasi untuk pengguna 582: ['Watchmen (2009)', 'Edge of Tomorrow (2014)', 'V for Vendetta (2006)', 'Batman Begins (2005)', 'Amazing Spider-Man, The (2012)', 'Matrix, The (1999)', 'The Hunger Games: Catching Fire (2013)', 'I Am Legend (2007)', 'Star Trek Into Darkness (2013)', 'Chronicle (2012)']
Pengguna 582 memiliki 1 hits.
Pengguna 215 memiliki 14 film relevan.
Rekomendasi untuk pengguna 215: ['Casino (1995)', 'Godfather, The (1972)', 'Godfather: Part II, The (1974)', 'American History X (1998)', 'Dog Day Afternoon (1975)', 'Bonnie and Clyde (1967)', 'Goodfellas (1990)', 'Once Were Wa

**Dari data yang disajikan, terlihat bahwa kinerja sistem rekomendasi ini, yang diukur dengan metrik Precision@k dan Recall@k pada k=10, masih sangat rendah.**

* Rata-rata Precision@10 sebesar 0.0380 menunjukkan bahwa, rata-rata, hanya sekitar 3.8% dari film yang direkomendasikan kepada pengguna adalah film yang relevan (film yang mereka sukai). Dengan kata lain, dari 10 film yang direkomendasikan, hanya sekitar 0.38 film (kurang dari satu) yang benar-benar relevan.

* Rata-rata Recall@10 sebesar 0.0285 menunjukkan bahwa, rata-rata, sistem hanya berhasil menemukan sekitar 2.85% dari total film relevan yang disukai pengguna. Ini berarti sistem gagal menemukan sebagian besar film relevan yang ada dalam data pengguna.

Meskipun model ini berhasil memberikan rekomendasi yang terlihat masuk akal secara tematis (misalnya, rekomendasi film-film aksi/sci-fi untuk Pengguna 582 dan 599, atau film kriminal untuk Pengguna 215), jumlah "hits" (film yang direkomendasikan dan juga relevan) sangat sedikit. Dari 32 pengguna yang dievaluasi, hanya 9 pengguna yang mendapatkan setidaknya satu film relevan di antara 10 rekomendasi teratas mereka.

**Analisis Lebih Lanjut**

Hasil ini menunjukkan bahwa ada beberapa area yang perlu diperbaiki:

* Keterbatasan Data: Jumlah pengguna dan film yang digunakan untuk evaluasi ini mungkin tidak cukup besar untuk memberikan gambaran yang akurat. Selain itu, sebaran data rating mungkin tidak merata, menyebabkan beberapa pengguna memiliki sedikit atau bahkan nol film relevan (seperti Pengguna 207 dan 496).

* Masalah Cold-Start: Sebagian besar pengguna dalam evaluasi ini (seperti Pengguna 428, 215, 461, 197, dst.) memiliki hits nol. Ini bisa menjadi indikasi masalah cold-start, di mana model tidak memiliki cukup data riwayat rating untuk pengguna tersebut, sehingga rekomendasinya tidak akurat.

* Bobot Model Hibrida: Bobot yang diberikan pada model Collaborative dan Content-based (jika ini adalah model hibrida) mungkin tidak optimal. Model mungkin terlalu condong ke salah satu sisi, sehingga gagal memanfaatkan kekuatan gabungan dari kedua pendekatan.

* Definisi "Relevan": Mungkin ada masalah dalam definisi "film relevan". Jika hanya film dengan rating 4 atau 5 yang dianggap relevan, model mungkin kesulitan jika sebagian besar rating pengguna berada di angka 3.

Secara keseluruhan, meskipun proyek ini berhasil membangun sebuah alur kerja rekomendasi, hasil evaluasi menunjukkan bahwa model ini belum siap untuk digunakan di lingkungan produksi. Diperlukan iterasi lebih lanjut pada model, data, dan strategi evaluasi untuk meningkatkan performa secara signifikan.

# **Implementasi Sistem Rekomendasi**

Proyek ini telah berhasil mengintegrasikan dua pendekatan rekomendasi yang berbeda untuk mengatasi keterbatasan masing-masing model.

**Content-based Filtering**: Model ini berfungsi dengan baik untuk mengatasi masalah cold-start—memberikan rekomendasi yang relevan kepada pengguna baru atau untuk film yang baru ditambahkan. Model ini menggunakan metadata film seperti genre dan tag untuk menemukan kemiripan. Kelemahan utamanya adalah ia cenderung merekomendasikan film yang terlalu mirip dengan apa yang sudah disukai pengguna, membatasi penemuan konten baru.

**Collaborative Filtering**: Menggunakan data rating dari pengguna, model ini mampu menemukan pola-pola tersembunyi dan mengidentifikasi film yang disukai oleh komunitas pengguna dengan selera serupa. Model ini sangat efektif dalam memberikan rekomendasi yang tidak terduga, atau serendipity. Namun, model ini tidak dapat berfungsi tanpa riwayat interaksi pengguna yang memadai, menjadikannya rentan terhadap masalah cold-start.

Dengan menggabungkan kedua model ini menjadi sistem hibrida, kita mendapatkan yang terbaik dari kedua dunia. Kami menggunakan model Collaborative Filtering untuk memprediksi film-film teratas yang mungkin disukai pengguna, lalu menggunakan model Content-based untuk memprioritaskan film-film tersebut berdasarkan kemiripan konten. Proses re-ranking ini memastikan bahwa rekomendasi tidak hanya akurat (berkat Collaborative) tetapi juga relevan dan personal (berkat Content-based).

## Strategi Bisnis

Sistem hibrida ini memiliki potensi besar untuk diintegrasikan ke dalam operasi bisnis. Berikut adalah beberapa strategi utama:

* Personalisasi Halaman Utama: Rekomendasi paling relevan akan ditampilkan di halaman utama, memastikan pengguna langsung disambut dengan konten yang mereka sukai. Ini dapat meningkatkan rata-rata sesi pengguna dan menurunkan tingkat bounce rate.

* Fitur "Lebih seperti ini": Di halaman detail film, pengguna akan melihat daftar film yang mirip secara tematis. Fitur ini dapat meminimalkan waktu yang dihabiskan pengguna untuk mencari konten dan mendorong mereka untuk terus menonton.

* Kampanye Pemasaran yang Ditargetkan: Model dapat digunakan untuk mengidentifikasi film-film yang mungkin disukai oleh pengguna yang tidak aktif. Rekomendasi yang dipersonalisasi dapat dikirim melalui email atau notifikasi untuk mendorong mereka kembali ke platform.

* Onboarding Pengguna Baru: Saat pengguna pertama kali mendaftar, mereka dapat diminta untuk memberi rating pada beberapa film. Sistem dapat menggunakan model Content-based untuk memberikan rekomendasi instan, menciptakan pengalaman yang menarik sejak awal.

## Kesimpulan

Secara keseluruhan, meskipun proyek ini berhasil membangun alur kerja rekomendasi, hasil evaluasi menunjukkan adanya kebutuhan mendesak untuk mengoptimalkan model, menyesuaikan bobot hibrida, dan mempertimbangkan strategi yang lebih efektif untuk mengatasi pengguna dengan riwayat rating yang minim.